# Correlation Analysis with Markup

This notebook performs data loading, preprocessing, and feature engineering similar to the training script, then analyzes the correlation between the engineered features and the 'markup' feature.

In [ ]:
import logging
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Setup data directory environment variable if needed (adjust path as necessary)
data_dir = os.path.abspath(os.path.join('..', '..', 'data')) # Adjust if your data dir is elsewhere
if os.path.exists(data_dir):
    os.environ['DATA_DIR'] = data_dir
    print(f"Set DATA_DIR to: {data_dir}")
else:
    print(f"Warning: Data directory not found at {data_dir}. Ensure DATA_DIR is set correctly.")

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Add src directory to Python path to import modules
module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    print(f"Added {module_path} to sys.path")

from gomat_markup_opt.data.preparation import (
    load_and_prepare_inventory_data,
    load_and_prepare_quote_lost_data,
    load_and_prepare_quote_order_data,
)
from gomat_markup_opt.preprocessing.cleanup import (
    cleanup_and_filter_inventory_data,
    cleanup_and_filter_quote_and_order_data,
)
from gomat_markup_opt.preprocessing.feature_engineering import engineer_features

# Define constants similar to the training script
TARGET_COLUMN = "target" # Represents quote acceptance (1) or rejection (0)
MARKUP_COLUMN = "markup"
MODEL_NUMERICAL_FEATURES = [
    "quantity",
    "seller_total",
    "markup",
    "seller_price_relative_to_market",
]
MODEL_CATEGORICAL_FEATURES = [
    "plant_category",
    "buyer_region",
    "qto",
]

# Set display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Load and Preprocess Data

In [ ]:
logger.info("--- Preparing Data ---")
prepared_quote_order_data = load_and_prepare_quote_order_data()
prepared_quote_lost_data = load_and_prepare_quote_lost_data()
prepared_inventory_data = load_and_prepare_inventory_data()

logger.info("--- Cleaning Prepared Data ---")
cleaned_quote_data = cleanup_and_filter_quote_and_order_data(prepared_quote_order_data, prepared_quote_lost_data)
cleaned_inventory_data = cleanup_and_filter_inventory_data(prepared_inventory_data)

logger.info("--- Engineering Features ---")
engineered_data = engineer_features(
    cleaned_quote_data, 
    cleaned_inventory_data, 
    TARGET_COLUMN, 
    MODEL_NUMERICAL_FEATURES, 
    MODEL_CATEGORICAL_FEATURES
)

logger.info(f"Engineered data shape: {engineered_data.shape}")
logger.info("Engineered data columns: %s", engineered_data.columns.tolist())
engineered_data.head()

## 2. List All Features

In [ ]:
all_features = engineered_data.columns.tolist()
print("All Features:")
for feature in all_features:
    print(f"- {feature}")

## 3. Plot Numerical Correlation with Markup

Calculate the correlation matrix for numerical features and plot the correlation of each feature with the 'markup'.

In [ ]:
# Select only numerical columns for correlation analysis
# Include the target column as well to see its correlation
numerical_cols_for_corr = engineered_data.select_dtypes(include=np.number).columns.tolist()

if MARKUP_COLUMN not in numerical_cols_for_corr:
    logger.warning(f"'{MARKUP_COLUMN}' not found in numerical columns. Cannot calculate correlations.")
else:
    logger.info(f"Calculating correlation for columns: {numerical_cols_for_corr}")
    correlation_matrix = engineered_data[numerical_cols_for_corr].corr()

    # Get correlation with the markup column
    markup_correlation = correlation_matrix[MARKUP_COLUMN].sort_values(ascending=False)

    print("\nCorrelation with Markup (Numerical Features):")
    print(markup_correlation)

    # Plotting the correlations with Markup
    plt.figure(figsize=(10, 8))
    sns.barplot(x=markup_correlation.values, y=markup_correlation.index, palette='viridis')
    plt.title(f'Numerical Feature Correlation with {MARKUP_COLUMN}')
    plt.xlabel('Correlation Coefficient')
    plt.ylabel('Features')
    plt.tight_layout()
    plt.show()

    # Optional: Plot the full correlation heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
    plt.title('Correlation Matrix of Numerical Features')
    plt.tight_layout()
    plt.show()

## 4. Visualize Markup Distribution by Categorical Features

Use box plots to understand how the distribution of 'markup' varies across different categories for each categorical feature.

In [ ]:
logger.info("--- Visualizing Markup vs Categorical Features ---")

categorical_features_in_data = [f for f in MODEL_CATEGORICAL_FEATURES if f in engineered_data.columns]

if MARKUP_COLUMN not in engineered_data.columns:
    logger.error(f"'{MARKUP_COLUMN}' column not found in the engineered data. Cannot create box plots.")
elif not categorical_features_in_data:
    logger.warning("No specified categorical features found in the engineered data.")
else:
    for cat_feature in categorical_features_in_data:
        # Check number of unique categories to avoid overly cluttered plots
        n_unique = engineered_data[cat_feature].nunique()
        if n_unique > 50:
            logger.warning(f"Feature '{cat_feature}' has {n_unique} unique values. Skipping box plot to avoid clutter.")
            # Optional: could plot top N categories instead
            # top_n = engineered_data[cat_feature].value_counts().nlargest(20).index
            # sns.boxplot(data=engineered_data[engineered_data[cat_feature].isin(top_n)], x=cat_feature, y=MARKUP_COLUMN)
            continue
            
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=engineered_data, x=cat_feature, y=MARKUP_COLUMN, palette='viridis')
        plt.title(f'Markup Distribution by {cat_feature}')
        plt.xlabel(cat_feature)
        plt.ylabel(MARKUP_COLUMN)
        plt.xticks(rotation=45, ha='right') # Rotate labels if they overlap
        plt.tight_layout()
        plt.show()
        logger.info(f"Generated box plot for {MARKUP_COLUMN} vs {cat_feature}")